In [1]:
# 데이터 프레임 관련 임포트
import numpy  as np
import pandas as pd

# 웹 관련 임포트
import urllib
import requests
from selenium import webdriver # version_3.14.1
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

# 기타
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

### 최종

In [3]:
driver = webdriver.Chrome('./driver_m1/chromedriver')
driver.get('https://www.elis.go.kr/allalr/allAlrList')
driver.maximize_window()

In [38]:
driver.close()

In [4]:
# 시도 선택
element = driver.find_element_by_id('ctpvCd')
element.send_keys('서울')
time.sleep(1)

# 시군구 > 본청 선택
element = driver.find_element_by_id('sggCd')
element.send_keys('본청')
time.sleep(1)

# 법규구분 > 조례 선택
button = driver.find_element_by_id('chk02')
driver.execute_script('arguments[0].click();', button)
time.sleep(1)

# 검색어 입력
searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
searchByName.clear()
searchByName.send_keys('도시계획 조례')
search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()

# 도시계획 조례 클릭
driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()

In [23]:
### 0. 임포트
# 데이터 프레임 관련 임포트
import numpy  as np
import pandas as pd

# 웹 관련 임포트
import urllib
import requests
from selenium import webdriver # version_3.14.1
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

# 기타
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

### 1. 필요한 정보 추출
# 1-1. 시도 정보 추출
def getSido():
    global sido_names
    sido_list_selectbox = driver.find_element_by_xpath('//*[@id="ctpvCd"]') # 셀렉트 박스 선택
    sido_list = sido_list_selectbox.find_elements_by_tag_name('option') # 셀렉트 박스 안에 있는 옵션 선택
    sido_names = [sido.text for sido in sido_list] # 시도명 리스트에 담기
    sido_names = sido_names[1:-1]

# 1-2. 선택 지역의 시군구 정보 추출
def getSigungu(sido):
    global sigungu_names
    sigungu_names = []

    # 시도 선택
    element = driver.find_element_by_id('ctpvCd')
    element.send_keys(sido)
    time.sleep(1)

    # 각 시도의 시군구 정보 추출
    sigungu_list_selectbox = driver.find_element_by_xpath('//*[@id="sggCd"]') # 셀렉트 박스 선택
    sigungu_list = sigungu_list_selectbox.find_elements_by_tag_name('option') # 셀렉트 박스 안에 있는 옵션 선택
    time.sleep(1)
    sigungu_name = [sigungu.text for sigungu in sigungu_list] # 시군구명 리스트에 담기
    sigungu_names.extend(sigungu_name)
    sigungu_names = sigungu_names[2:]

# 1-3. id값 으로 버튼 클릭
def clickChecktBox():
    global checked
    checked = []
    num = 0
    driver.switch_to_window(driver.window_handles[1])
    time.sleep(1)
    
    while True:
        try:
            # 조 목록 가져오기
            num += 1
            jo = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]').text
            time.sleep(1)

            # 조 목록에 용적률, 건폐율이 포함된 조항만 체크박스 선택
            if '용도지역' in jo and '용적률' in jo:
                joid = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]/label').get_attribute('for') # 해당 조항의 id값 가져오기
                button = driver.find_element_by_id(joid) # id값 선택
                driver.execute_script('arguments[0].click();', button) # 체크박스 클릭
                checked.append(joid)
                time.sleep(1)
            elif '용도지역' in jo and '건폐율' in jo:
                joid = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]/label').get_attribute('for') # 해당 조항의 id값 가져오기
                button = driver.find_element_by_id(joid) # id값 선택
                driver.execute_script('arguments[0].click();', button) # 체크박스 클릭
                checked.append(joid)
                time.sleep(1)
            else:
                pass
        except:
            break

# 1-4. 용적률, 건폐율 파일 다운로드
def downloadFile():
    if len(checked) > 0:
        # txt 저장
        driver.find_element_by_xpath('/html/body/div/div/div/div[1]/a[1]').click()
        time.sleep(4)

        # 창 닫기, 검색 창으로 전환
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
        driver.back()
    else:
        # 창 닫기, 검색 창으로 전환
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
        driver.back()

### 2. txt 다운로드
# 2-1. 도시계획 조례 txt 다운로드 (전지역)
def getTxt(sido):
    sido = sido
    # 시도 선택
    element = driver.find_element_by_id('ctpvCd')
    element.send_keys(sido)
    time.sleep(1)
    
    # 시군구 > 본청 선택
    element = driver.find_element_by_id('sggCd')
    element.send_keys('본청')

    # 법규구분 > 조례 선택
    button = driver.find_element_by_id('chk02') # 조례 선택
    driver.execute_script('arguments[0].click();', button) # 클릭
    time.sleep(1)

    # 검색어 입력
    searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
    time.sleep(1)
    searchByName.clear()
    searchByName.send_keys('도시계획 조례')
    search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()
    
    # 도시계획 조례 클릭
    driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()
    time.sleep(1)

    # txt 저장 버튼 클릭
    driver.find_element_by_xpath('//*[@id="btnSave"]').click()
    time.sleep(2)
    driver.switch_to_window(driver.window_handles[1]) # 새창 전환
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/span/label').click() # 전체선택 해제
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/span/label').click() # 전체선택 해제
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[3]/div/span/label').click() # 부칙선택 해제

    # 목록 이름에 용적률, 건폐율이 포함된 조항만 선택
    clickChecktBox()
    
    # 파일 다운로드
    downloadFile()
    
    # 각 시군구 파일 다운로드
    if len(checked) == 0:
        getTxtSep(sido)
    else:
        pass

# 2-2. 도시계획 조례 txt 다운로드 (각 시군구 별)
def getTxtSep(sido):
    getSigungu(sido)
    for i in sigungu_names:
        # 시군구 > 시군구 선택
        element = driver.find_element_by_id('sggCd')
        element.send_keys(i)
        time.sleep(1)

        # 법규구분 > 조례 선택
        button = driver.find_element_by_id('chk02')
        driver.execute_script('arguments[0].click();', button)
        time.sleep(1)

        # 검색어 입력
        searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
        time.sleep(1)
        searchByName.clear()
        searchByName.send_keys('도시계획 조례')
        search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()

        # 도시계획 조례 클릭
        try:
            driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()
            time.sleep(1)

            # txt 저장 버튼 클릭
            driver.find_element_by_xpath('//*[@id="btnSave"]').click()
            time.sleep(2)
            driver.switch_to_window(driver.window_handles[1]) # 새창 전환
            driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/span/label').click() # 전체선택 해제
            driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/span/label').click() # 전체선택 해제
            driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[3]/div/span/label').click() # 부칙선택 해제

            # 목록 이름에 용적률, 건폐율이 포함된 조항만 선택
            clickChecktBox()

            # 파일 다운로드
            downloadFile()
        except:
            pass

In [30]:
### 4. 전 지역 도시계획 조례 txt 다운로드 실행
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://www.elis.go.kr/allalr/allAlrList')
driver.maximize_window()
getSido()

for sido in tqdm(sido_names):
    getTxt(sido)

driver.close()

  0%|                                                     | 0/2 [00:04<?, ?it/s]


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=105.0.5195.102)


In [24]:
getSido()

In [27]:
sido_names[8]

'경기'

In [ ]:
getTxt(sido_names[8])

/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_1019/40413154.py:128: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1]) # 새창 전환
/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_1019/40413154.py:51: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_1019/40413154.py:93: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_1019/40413154.py:174: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1]) # 새창 전환
/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_1019/40413154.py:88: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])


In [122]:
### 4. 전 지역 도시계획 조례 txt 다운로드 실행
driver = webdriver.Chrome('./driver_m1/chromedriver')
driver.get('https://www.elis.go.kr/allalr/allAlrList')
driver.maximize_window()
getSido()

for sido in tqdm(sido_names):
    getTxt(sido)

driver.close()

  0%|                                                    | 0/17 [00:00<?, ?it/s]/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_2432/2647038236.py:126: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1]) # 새창 전환
/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_2432/2647038236.py:50: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_2432/2647038236.py:87: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
 47%|████████████████████▏                      | 8/17 [17:24<19:04, 127.13s/it]/var/folders/r8/5fffyvfd0713h07trp3wxj700000gn/T/ipykernel_2432/2647038236.py:92: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
 47%|████████████████████▏                      | 8/17 [18:16<20:33, 13

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=105.0.5195.102)


### 파일 한 번에 불러오기

###### test

ver1 (try-except)

In [28]:
from glob import glob

# 변수 생성
tmp_raw = []
re_list = []
df_buildingCoverage = pd.DataFrame()
df_floorAreaRatio = pd.DataFrame()

# txt to dataframe
for file_name in glob('C:\Users\duPar\Downloads\*도시계획*.txt'):
    with open(file_name, encoding='utf-8') as f:
        lines = f.readlines()
        df = pd.DataFrame(lines, columns=[file_name.split(' ')[0][2:]]).drop_duplicates(keep=False).reset_index(drop=True)
        df[file_name.split(' ')[0][2:]] = df[file_name.split(' ')[0][2:]].str.replace('\n','')
        
        try:
            # 해당 인덱스부터 15개 가져오기 -> 건폐율, 용적률 순서
            idx = df[df[file_name.split(' ')[0][2:]].str.contains('제1종전용주거지역')].index            
            
            # 건폐율, 용적률 데이터프레임 나누기
            df_bc = df.loc[idx[0]:][0:16].reset_index(drop=True) # 건폐율
            df_far = df.loc[idx[1]:][0:16].reset_index(drop=True) # 용적률

            # 데이터프레임 합치기
            df_buildingCoverage = pd.concat([df_buildingCoverage, df_bc], axis=1).reset_index(drop=True)
            df_floorAreaRatio = pd.concat([df_floorAreaRatio, df_far], axis=1).reset_index(drop=True)
            
        except:
            re_list.append(file_name.split(' ')[0][2:])
            pass

# 각 시군구 txt 파일 수집
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://www.elis.go.kr/allalr/allAlrList')
downloadSep(re_list)
driver.close()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.102)


In [39]:
from glob import glob

# 변수 생성
tmp_raw = []
re_list = []
df_buildingCoverage = pd.DataFrame()
df_floorAreaRatio = pd.DataFrame()

In [41]:
glob('C:\\Users\duPar\Downloads\*도시계획*.txt')

['C:\\Users\\duPar\\Downloads\\강원도 도시계획 조례 (19).txt',
 'C:\\Users\\duPar\\Downloads\\경기도 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\경상남도 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\경상북도 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\광주광역시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\대구광역시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\대전광역시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\부산광역시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\서울특별시 도시계획 조례 (7).txt',
 'C:\\Users\\duPar\\Downloads\\세종특별자치시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\울산광역시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\인천광역시 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\전라남도 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\전라북도 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\제주특별자치도 도시계획 조례 (12).txt',
 'C:\\Users\\duPar\\Downloads\\충청남도 도시계획 조례 (4).txt',
 'C:\\Users\\duPar\\Downloads\\충청북도 도시계획 조례 (4).txt']

In [42]:
file_name = 'C:\\Users\\duPar\\Downloads\\강원도 도시계획 조례 (19).txt'

In [57]:
# txt to dataframe
with open(file_name, encoding='utf-8') as f:
    lines = f.readlines()
    df = pd.DataFrame(lines, columns=[file_name.split(' ')[0].split('\\')[-1]]).drop_duplicates(keep=False).reset_index(drop=True)
    df[file_name.split(' ')[0].split('\\')[-1]] = df[file_name.split(' ')[0].split('\\')[-1]].str.replace('\n','')

In [58]:
df

,강원도
0,제6조의3(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 삭제<202...
1,② 영 제46조제1항제2호에 따른 공공시설등의 설치비용과 부지가액의 산정방법은 ...
2,1. 공공시설등의 설치비용은 「지방자치단체를 당사자로 하는 계약에 관한 법률 시...
3,2. 부지가액은 국토교통부장관이 정하는 요건을 갖춘 「감정평가 및 감정평가사에 ...
4,3. 공공시설 등의 설치비용 및 부지가액의 산정은 건축허가 신청일을 기준으로 한...
5,4. 공공시설등의 설치이행을 담보하기 위하여 이행보증금을 예치하게 할 수 있으며...
6,5. 제1호 및 제2호에 소요되는 비용은 공공시설등을 설치·제공하는 자가 부담하...
7,③ 영 제46조제11항에 따라 공공시설등의 부지를 제공하거나 공공시설등을 설치하...
8,1. 토지이용계획 및 기반시설 설치계획
9,2. 건축 및 주택건설 계획


In [60]:
# 해당 인덱스부터 15개 가져오기 -> 건폐율, 용적률 순서
idx = df[df[file_name.split(' ')[0].split('\\')[-1]].str.contains('제1종전용주거지역')].index

In [95]:
type(idx)

builtin_function_or_method

In [93]:
idx = df['강원도'][0].index

In [88]:
idx = 1

In [94]:
if bool(idx.size > 0) == False:
    pass
else:
    # 건폐율, 용적률 데이터프레임 나누기
    df_bc = df.loc[idx[0]:][0:16].reset_index(drop=True) # 건폐율
    df_far = df.loc[idx[1]:][0:16].reset_index(drop=True) # 용적률
    
    # 데이터프레임 합치기
    df_buildingCoverage = pd.concat([df_buildingCoverage, df_bc], axis=1).reset_index(drop=True)
    df_floorAreaRatio = pd.concat([df_floorAreaRatio, df_far], axis=1).reset_index(drop=True)
    re_list.append(file_name.split(' ')[0][2:])

AttributeError: 'builtin_function_or_method' object has no attribute 'size'

In [85]:
re_list = 

[]

In [ ]:
# 각 시군구 txt 파일 수집
downloadSep(re_list)

ver2 (if)

In [139]:
from glob import glob

# txt to dataframe
tmp_raw = []
df_buildingCoverage = pd.DataFrame()
df_floorAreaRatio = pd.DataFrame()
re_list = []

for file_name in glob('./*도시계획*.txt'):
    with open(file_name, encoding='utf-8') as f:
        lines = f.readlines()
        df = pd.DataFrame(lines, columns=[file_name.split(' ')[0][2:]]).drop_duplicates(keep=False).reset_index(drop=True)
        df[file_name.split(' ')[0][2:]] = df[file_name.split(' ')[0][2:]].str.replace('\n','')
        
        # try:
        # 해당 인덱스부터 15개 가져오기 -> 건폐율, 용적률 순서
        idx = df[df[file_name.split(' ')[0][2:]].str.contains('제1종전용주거지역')].index

        if idx.size == 0:
            re_list.append(file_name.split(' ')[0][2:])

            # 페이지 오픈
            driver = webdriver.Chrome('./driver/chromedriver.exe')
            driver.get('https://www.elis.go.kr/allalr/allAlrList')

            # 시도, 시군구 추출
            getSigungu()

            # txt file 수집
            for sigungu in tqdm(sigungu_names[1:-1]):
                getTxtSep(sido_names[1:-1])

            driver.close()
            pass
        else:
            # 건폐율, 용적률 데이터프레임 나누기
            df_bc = df.loc[idx[0]:][0:16].reset_index(drop=True) # 건폐율
            df_far = df.loc[idx[1]:][0:16].reset_index(drop=True) # 용적률

            # 데이터프레임 합치기
            df_buildingCoverage = pd.concat([df_buildingCoverage, df_bc], axis=1).reset_index(drop=True)
            df_floorAreaRatio = pd.concat([df_floorAreaRatio, df_far], axis=1).reset_index(drop=True)
#         except:
#             re_list.append(file_name.split(' ')[0][2:])
            
#             # 전 지역 txt 파일 다운로드
#             getTxtSep(sido)
            
#             pass

  6%|████▉                                                                              | 1/17 [00:50<13:34, 50.89s/it]


KeyboardInterrupt: 

In [150]:
file_name

'.\\강원도 도시계획 조례.txt'

In [147]:
with open(file_name, encoding='utf-8') as f:
    lines = f.readlines()
    df = pd.DataFrame(lines, columns=[file_name.split(' ')[0][2:]]).drop_duplicates(keep=False).reset_index(drop=True)
    df[file_name.split(' ')[0][2:]] = df[file_name.split(' ')[0][2:]].str.replace('\n','')

In [148]:
df

,강원도
0,제6조의3(도시지역 내 지구단위계획구역에서의 건폐율 등의 완화적용) ① 삭제<202...
1,② 영 제46조제1항제2호에 따른 공공시설등의 설치비용과 부지가액의 산정방법은 ...
2,1. 공공시설등의 설치비용은 「지방자치단체를 당사자로 하는 계약에 관한 법률 시...
3,2. 부지가액은 국토교통부장관이 정하는 요건을 갖춘 「감정평가 및 감정평가사에 ...
4,3. 공공시설 등의 설치비용 및 부지가액의 산정은 건축허가 신청일을 기준으로 한...
5,4. 공공시설등의 설치이행을 담보하기 위하여 이행보증금을 예치하게 할 수 있으며...
6,5. 제1호 및 제2호에 소요되는 비용은 공공시설등을 설치·제공하는 자가 부담하...
7,③ 영 제46조제11항에 따라 공공시설등의 부지를 제공하거나 공공시설등을 설치하...
8,1. 토지이용계획 및 기반시설 설치계획
9,2. 건축 및 주택건설 계획


In [154]:
df[df[file_name.split(' ')[0][2:]].str.contains('제1종전용주거지역')]

,강원도


In [152]:
idx

Int64Index([], dtype='int64')

### txt to Dataframe

###### 방법1 (text read)

In [6]:
# read txt file
f = open('./서울특별시 도시계획 조례.txt', 'rt', encoding='utf-8')
data = f.read()

In [19]:
data

'\n제1장 총칙\n\n제2장 광역도시계획 및 도시기본계획\n\n제3장 도시관리계획의 입안\n\n제4장 용도지구의 지정\n\n제5장 도시계획시설의 관리\n\n제6장 지구단위계획\n\n제7장 개발행위의 허가\n\n제8장 용도지역ㆍ용지지구 및 용도구역안에서의 행위제한 <개정 2011. 7. 28.>\n\n제1절 용도지역안에서의 행위제한\n\n제2절 경관지구안에서의 건축제한\n\n제3절 보호지구안에서의 건축제한 <신설 2018. 10. 4.>\n\n제4절 그 밖의 용도지구안에서의 건축제한 <신설 2018. 10. 4.>\n\n제5절 건폐율 및 용적률 <신설 2018. 10. 4.>\n제54조(용도지역안에서의 건폐율) ① 법 제77조 및 영 제84조제1항에 따라 용도지역별 건폐율은 다음 각 호의 비율 이하로 한다. <개정 2008. 7. 30.>\n  1. 제1종전용주거지역：50퍼센트\n  2. 제2종전용주거지역：40퍼센트\n  3. 제1종일반주거지역：60퍼센트\n  4. 제2종일반주거지역：60퍼센트\n  5. 제3종일반주거지역：50퍼센트\n  6. 준주거지역：60퍼센트\n  7. 중심상업지역：60퍼센트\n  8. 일반상업지역：60퍼센트\n  9. 근린상업지역：60퍼센트\n  10. 유통상업지역：60퍼센트\n  11. 전용공업지역：60퍼센트\n  12. 일반공업지역：60퍼센트\n  13. 준공업지역：60퍼센트\n  14. 보전녹지지역：20퍼센트\n  15. 생산녹지지역：20퍼센트\n  16. 자연녹지지역：20퍼센트\n  ② 법 제77조제3항 및 영 제84조제4항에 따라 다음 각 호의 지역 안에서의 건폐율은 제1항에도 불구하고 다음 각 호의 비율 이하로 한다. <개정 2008. 7. 30., 2011. 7. 28., 2012. 11. 1., 2016. 9. 29., 2019. 7. 18., 2020. 7. 16.>\n  1. 취락지구 : 60퍼센트(집단취락지구에 대해서는 「개발제한구역의 지정 및 관리에 관한 특별조치법령」 이 정하는 바에 따른다)\n  2. 「자연공원법

###### 방법2 (txt to dataframe)

In [35]:
# txt to dataframe
df = pd.read_csv('./서울특별시 도시계획 조례.txt', sep='\t', header=None)

In [36]:
df.head(20)

,0
0,제1장 총칙
1,제2장 광역도시계획 및 도시기본계획
2,제3장 도시관리계획의 입안
3,제4장 용도지구의 지정
4,제5장 도시계획시설의 관리
5,제6장 지구단위계획
6,제7장 개발행위의 허가
7,제8장 용도지역ㆍ용지지구 및 용도구역안에서의 행위제한 <개정 2011. 7. 28.>
8,제1절 용도지역안에서의 행위제한
9,제2절 경관지구안에서의 건축제한


In [336]:
# 55조(용적률) 찾기
df[df[0].str.contains('퍼센트')].head(20)

,0
14,1. 제1종전용주거지역：50퍼센트
15,2. 제2종전용주거지역：40퍼센트
16,3. 제1종일반주거지역：60퍼센트
17,4. 제2종일반주거지역：60퍼센트
18,5. 제3종일반주거지역：50퍼센트
19,6. 준주거지역：60퍼센트
20,7. 중심상업지역：60퍼센트
21,8. 일반상업지역：60퍼센트
22,9. 근린상업지역：60퍼센트
23,10. 유통상업지역：60퍼센트


###### 방법3 (doc document)

In [45]:
# !pip install python-docx

     ---------------------------------------- 5.6/5.6 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=024d0d997fff1fca25290e949cf49b65e4d90431a5d772d6f58dd216a98494f4
  Stored in directory: c:\users\dupar\appdata\local\pip\cache\wheels\32\b8\b2\c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [50]:
# import docx NOT python-docx
from docx import Document
# create an instance of a word document
doc = docx.Document()

In [55]:
# doc 파일 불러오기
# doc = Document('./서울특별시 도시계획 조례(서울특별시조례)(제8435호)(20220711).doc')
doc = Document(r'C:\oasis\서울특별시 도시계획 조례(서울특별시조례)(제8435호)(20220711).doc')

PackageNotFoundError: Package not found at 'C:\oasis\서울특별시 도시계획 조례(서울특별시조례)(제8435호)(20220711).doc'